# Opening the files

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
import src.lib.modules as modules
import warnings

warnings.filterwarnings("ignore")
%matplotlib inline
np.set_printoptions(precision=5)
pd.set_option("display.precision", 5,
              'display.max_rows', None,
              'display.max_columns', None,
              'display.width', None,
              'display.max_colwidth', None)
# change the colors
# plt.style.use('ggplot')
plt.style.use('seaborn-whitegrid')

# -----------------------------------------------------

# NOTE: Defining the columns that we are interested in
x = 'Oscillation strain'
y = ['Storage modulus', 'Loss modulus', 'Tan(delta)']  # should be a list

# -----------------------------------------------------

# Defining specific parameters for the plots
interested_in = y + [x]
DATA_PATH = modules.DATA_PATH
snsc = modules.get_color_palette(y)
sns.set_palette(snsc)

# Opening the files
dfs, files, common_name = modules.load_files_df(interested_in=interested_in)


# Curve fitting Storage M.

$$
f(x) = \frac{L}{1 + exp(-k * (x - x_0))} + b
$$

- **L** is responsible for scaling the output range from [0,1] to [0,L]
- **b** adds bias to the output and changes its range from [0,L] to [b,L+b]
- **k** is responsible for scaling the input, which remains in (-inf,inf)
- **x0** is the point in the middle of the Sigmoid, i.e. the point where Sigmoid should originally output the value 1/2
if $x=x_0$, we get $$\frac{1}{1+exp(0)} = 0.5$$

In [ ]:
def sigmoid(x, L, x0, k, b):
    y = L / (1 + np.exp(-k * (x - x0))) + b
    return (y)


filtered_df = av_df[(av_df['Temperature'] > start_point)].copy()
x_filtered = filtered_df.loc[:, 'Temperature'].to_numpy().astype(np.float16)
y1_filtered = filtered_df.loc[:, 'Storage modulus'].to_numpy().astype(np.float16)

y1_log = np.log10(y1_filtered)

p0 = [max(y1_filtered), np.median(x_filtered), 1, min(y1_filtered)]

popt, pcov = curve_fit(sigmoid, x_filtered, y1_log, p0, method='lm')

## 5.1. Looking at the fitted curve

In [ ]:
# change the colors
plt.style.use('ggplot')
plt.style.use('seaborn-paper')
plt.style.use('seaborn-whitegrid')

L_opt, x0_opt, k_opt, b_opt = popt

x_model = np.linspace(min(x_filtered), max(x_filtered), 100)
y_model = sigmoid(x_model, L_opt, x0_opt, k_opt, b_opt)

fig, ax = plt.subplots(dpi=150)

ax.scatter(x_filtered, y1_log, color=sns_blue, marker='o', s=20)
ax.plot(x_model, y_model, color=sns_red)

plt.show()

## 5.2. Looking at covariance matrix
The diagonals provide the variance of the parameter estimate.
One standard deviation errors on the parameters:

In [ ]:
perr = np.sqrt(np.diag(pcov))
labels = ['L', 'x0', 'k', 'b']
sns.barplot(x=labels, y=perr);

The result is 4x4 heat map. The main diagonal is related to the error (from top left to bottom right) for L, x0, k, b.

In [ ]:
fig, ax = plt.subplots(figsize=[6, 6], dpi=100)

ax.grid(which='minor', color='black', linestyle='-', linewidth=1, alpha=1)
ax.grid(which='major', alpha=0)

im = ax.imshow(error_data := np.log(np.abs(pcov)), cmap=sns.cubehelix_palette(as_cmap=True, reverse=True))
# plt.cm.inferno)

# Major ticks
ax.set_yticks(np.arange(0, 4, 1))
ax.set_xticks(np.arange(0, 4, 1))

# Labels for major ticks
ax.set_yticklabels(labels, fontsize=12)
ax.set_xticklabels(labels, fontsize=12)

# setting minor ticks for grid
ax.set_xticks(np.arange(-.5, 3.5, 1), minor=True, alpha=0)
ax.set_yticks(np.arange(-.5, 3.5, 1), minor=True, alpha=0)

# Creating a more advanced color map
divider = make_axes_locatable(plt.gca())
cax = divider.append_axes("right", "4%", pad="2%")
cbar = plt.colorbar(im, cax=cax)
cbar.ax.set_ylabel(ylabel="Error", rotation=90, fontsize=12)
cbar.ax.set_yticklabels(labels=np.arange(round(error_data.min(), 1), round(error_data.max(), 1), 0.5), fontsize=12)
cbar.minorticks_on()

plt.show()
